**Milestone 3: Route, Cancellation, and Seasonal Insights**

Week 5: Route and Airport-Level Analysis


In [0]:
!pip install folium
import folium

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import pandas as pd


df = pd.read_csv("/Workspace/Users/tulasivaliveti123@gmail.com/flights_preprocessed.csv")

# Show first 5 rows
df.head()

,DayOfWeek,Date,DepTime,ArrTime,CRSArrTime,UniqueCarrier,Airline,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Org_Airport,Dest,Dest_Airport,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,DepHour,DepMinute,DepDatetime,Month,Hour,Route,DepDateTime,ArrDateTime,CRSArrDateTime
0,Thursday,2019-01-03,1829,1959,1925,WN,Southwest Airlines Co.,3920,N464WN,90,90,77,34,34,IND,Indianapolis International Airport,BWI,Baltimore-Washington International Airport,515,3,10,0,N,0,2,0,0,0,32,18,29,2019-01-03 18:29:00,1,18,IND-BWI,2019-01-03 18:29:00,2019-01-03 19:59:00,2019-01-03 19:25:00
1,Thursday,2019-01-03,1937,2037,1940,WN,Southwest Airlines Co.,509,N763SW,240,250,230,57,67,IND,Indianapolis International Airport,LAS,McCarran International Airport,1591,3,7,0,N,0,10,0,0,0,47,19,37,2019-01-03 19:37:00,1,19,IND-LAS,2019-01-03 19:37:00,2019-01-03 20:37:00,2019-01-03 19:40:00
2,Thursday,2019-01-03,1644,1845,1725,WN,Southwest Airlines Co.,1333,N334SW,121,135,107,80,94,IND,Indianapolis International Airport,MCO,Orlando International Airport,828,6,8,0,N,0,8,0,0,0,72,16,44,2019-01-03 16:44:00,1,16,IND-MCO,2019-01-03 16:44:00,2019-01-03 18:45:00,2019-01-03 17:25:00
3,Thursday,2019-01-03,1452,1640,1625,WN,Southwest Airlines Co.,675,N286WN,228,240,213,15,27,IND,Indianapolis International Airport,PHX,Phoenix Sky Harbor International Airport,1489,7,8,0,N,0,3,0,0,0,12,14,52,2019-01-03 14:52:00,1,14,IND-PHX,2019-01-03 14:52:00,2019-01-03 16:40:00,2019-01-03 16:25:00
4,Thursday,2019-01-03,1323,1526,1510,WN,Southwest Airlines Co.,4,N674AA,123,135,110,16,28,IND,Indianapolis International Airport,TPA,Tampa International Airport,838,4,9,0,N,0,0,0,0,0,16,13,23,2019-01-03 13:23:00,1,13,IND-TPA,2019-01-03 13:23:00,2019-01-03 15:26:00,2019-01-03 15:10:00


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 484551 entries, 0 to 484550
Data columns (total 38 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DayOfWeek          484551 non-null  object
 1   Date               484551 non-null  object
 2   DepTime            484551 non-null  int64 
 3   ArrTime            484551 non-null  int64 
 4   CRSArrTime         484551 non-null  int64 
 5   UniqueCarrier      484551 non-null  object
 6   Airline            484551 non-null  object
 7   FlightNum          484551 non-null  int64 
 8   TailNum            484551 non-null  object
 9   ActualElapsedTime  484551 non-null  int64 
 10  CRSElapsedTime     484551 non-null  int64 
 11  AirTime            484551 non-null  int64 
 12  ArrDelay           484551 non-null  int64 
 13  DepDelay           484551 non-null  int64 
 14  Origin             484551 non-null  object
 15  Org_Airport        484551 non-null  object
 16  Dest               4

In [0]:
import pandas as pd
import plotly.express as px

# Load dataset
df = pd.read_csv("/Workspace/Users/tulasivaliveti123@gmail.com/flights_preprocessed.csv")

# Create route column if not exists
if 'Route' not in df.columns:
    df['Route'] = df['Origin'] + "-" + df['Dest']

# Top 10 busiest routes
top_routes = df['Route'].value_counts().head(10).reset_index()
top_routes.columns = ['Route', 'Count']

# Plotly bar chart
fig = px.bar(
    top_routes,
    x='Route',
    y='Count',
    text='Count',
    title='Top 10 Busiest Origin-Destination Pairs',
    color='Count',
    color_continuous_scale='Plasma'
)
fig.update_traces(textposition='outside')
fig.show()


In [0]:
import numpy as np

# Calculate average delay per origin airport
airport_delay = df.groupby('Origin')['ArrDelay'].mean().reset_index()

fig = px.density_heatmap(
    airport_delay,
    x='Origin',
    y='ArrDelay',
    nbinsx=len(airport_delay),
    title='Average Arrival Delay by Origin Airport',
    color_continuous_scale='RdYlBu_r'
)
fig.show()


In [0]:
route_delay = df.groupby('Route')['ArrDelay'].mean().reset_index()

fig = px.density_heatmap(
    route_delay,
    x='Route',
    y='ArrDelay',
    title='Average Delay per Route',
    color_continuous_scale='Viridis'
)
fig.show()


In [0]:
#Example lookup table (you can extend this)
airport_coords = {
    'ATL': (33.6407, -84.4277),
    'LAX': (33.9416, -118.4085),
    'ORD': (41.9742, -87.9073),
    'DFW': (32.8998, -97.0403),
    'DEN': (39.8561, -104.6737),
    'JFK': (40.6413, -73.7781),
    'SFO': (37.6213, -122.3790),
    'SEA': (47.4502, -122.3088),
    'MIA': (25.7959, -80.2870),
    'LAS': (36.0840, -115.1537),
    'YYZ': (43.6777, -79.6248)   
}



# Map coordinates to each row
df['Latitude'] = df['Org_Airport'].map(lambda x: airport_coords.get(x, (None, None))[0])
df['Longitude'] = df['Org_Airport'].map(lambda x: airport_coords.get(x, (None, None))[1])

In [0]:
# Map coordinates to each row
df['Latitude'] = df['Org_Airport'].map(lambda x: airport_coords.get(x, (None, None))[0])
df['Longitude'] = df['Org_Airport'].map(lambda x: airport_coords.get(x, (None, None))[1])
import numpy as np

# Assign random coordinates for visualization
unique_airports = df['Org_Airport'].unique()
coords = {a: (np.random.uniform(25, 49), np.random.uniform(-125, -67)) for a in unique_airports}

df["Latitude"] = df["Org_Airport"].map(lambda x: coords[x][0])
df["Longitude"] = df["Org_Airport"].map(lambda x: coords[x][1])

In [0]:
!pip install branca

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import pandas as pd
import folium
from branca.colormap import LinearColormap


# Ensure necessary columns
required_cols = ['Origin', 'Org_Airport', 'Latitude', 'Longitude', 'ArrDelay']
assert all(col in df.columns for col in required_cols), "Dataset missing one or more required columns!"

# Aggregate: flight count + average delay
airport_stats = (
    df.groupby(['Origin', 'Org_Airport', 'Latitude', 'Longitude'])
    .agg(Count=('Origin', 'size'), AvgDelay=('ArrDelay', 'mean'))
    .reset_index()
)

# Create color scale (green → yellow → red)

colormap =LinearColormap(
    colors=['green', 'yellow', 'red'],
    vmin=airport_stats['AvgDelay'].min(),
    vmax=airport_stats['AvgDelay'].max(),
    caption="Average Arrival Delay (minutes)"
)
colormap.caption = "Average Arrival Delay (minutes)"

airport_stats.head()


,Origin,Org_Airport,Latitude,Longitude,Count,AvgDelay
0,ABE,Lehigh Valley International Airport,36.873230,-103.899294,141,86.921986
1,ABI,Abilene Regional Airport,47.341543,-115.869256,157,100.898089
2,ABQ,Albuquerque International Sunport,45.281507,-93.602815,3026,50.931923
3,ABY,Southwest Georgia Regional Airport,45.821925,-70.217224,103,66.427184
4,ACT,Waco Regional Airport,46.631713,-114.949786,138,61.608696


In [0]:
!pip install reverse_geocoder

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 43.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for reverse_geocoder: filename=reverse_geocoder-1.5.1-py3-none-any.whl size=2268069 sha256=2a9046b09b1e47802b434cd8e80d40a8dab57a10124bf43c7077812af1e08684
  Stored in directory: /home/spark-c717a06f-2322-4733-9792-0d/.cache/pip/wheels/11/e1/67/6e47f0ad41ea1843d37e1fbe79c6074744a1f4aace641cf800
Successfully built reverse_geocoder
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import reverse_geocoder as rg
import pandas as pd

# Assuming df is your dataset
# Example: df = pd.read_csv("flights.csv")

# Take unique airports with coordinates
airports = df[['Origin', 'Latitude', 'Longitude']].drop_duplicates().dropna()

# Function to get country name and continent
def get_location_info(lat, lon):
    try:
        results = rg.search((lat, lon))
        country = results[0]['cc']    # country code (like 'US', 'IN', 'GB')
        name = results[0]['name']     # nearest city name
        admin = results[0]['admin1']  # state/province
        return pd.Series([country, name, admin])
    except:
        return pd.Series([None, None, None])

# Apply reverse geocoding
airports[['CountryCode', 'City', 'State']] = airports.apply(
    lambda x: get_location_info(x['Latitude'], x['Longitude']), axis=1
)

# Map country codes to continent
continent_map = {
    'NA': ['US', 'CA', 'MX'],
    'SA': ['BR', 'AR', 'CL', 'CO', 'PE', 'VE'],
    'EU': ['GB', 'FR', 'DE', 'NL', 'ES', 'IT', 'RU'],
    'AF': ['ZA', 'NG', 'EG', 'KE', 'MA', 'ET'],
    'AS': ['IN', 'CN', 'JP', 'AE', 'SG', 'TH', 'KR'],
    'OC': ['AU', 'NZ', 'FJ', 'PG']
}

def get_continent(code):
    for cont, countries in continent_map.items():
        if code in countries:
            return cont
    return 'Unknown'

airports['Continent'] = airports['CountryCode'].apply(get_continent)

# Merge back to original dataset
df = df.merge(airports[['Origin', 'CountryCode', 'Continent']], on='Origin', how='left')

# Show unique countries in your dataset
print("🌍 Countries in dataset:")
print(airports['CountryCode'].unique())

# Optional: show counts by continent
print("\n🌎 Flights by Continent:")
print(airports['Continent'].value_counts())


Loading formatted geocoded file...


I0000 00:00:1762178692.425044    2477 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1762178692.446261    2477 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1762178692.482822    2477 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1762178692.556480    2477 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1762178692.741963    2477 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1762178692.786450    2477 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1762178692.833675    2477 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1762178692.862817    2477 fork_posix.cc:77] Other threads are currently calling into gRPC, s

🌍 Countries in dataset:
['US' 'CA' 'MX' 'BS' 'CU' 'BM']

🌎 Flights by Continent:
Continent
NA         259
Unknown     15
Name: count, dtype: int64


In [0]:

import folium
from folium.plugins import HeatMap

# Aggregate total flights per airport
airport_traffic = df.groupby(['Origin', 'Org_Airport', 'Latitude', 'Longitude']).size().reset_index(name='Count')

# Initialize folium map
m = folium.Map(location=[39.8283, -98.5795], zoom_start=4, tiles='CartoDB positron')

# Add circles for busiest airports
for _, row in airport_traffic.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=row['Count'] / airport_traffic['Count'].max() * 20,
        popup=f"{row['Org_Airport']} ({row['Origin']}) - {row['Count']} flights",
        color='blue',
        fill=True,
        fill_opacity=0.6
    ).add_to(m)

m.save('busiest_airports.html')
m


In [0]:
m_na = folium.Map(location=[39.5, -98.35], zoom_start=4, tiles='CartoDB positron')

for _, row in airport_stats.iterrows():
    if 15 < row['Latitude'] < 72 and -170 < row['Longitude'] < -30:
        folium.CircleMarker(
            location=[row['Latitude'], row['Longitude']],
            radius=max(3, row['Count'] / airport_stats['Count'].max() * 20),
            color=colormap(row['AvgDelay']),
            fill=True,
            fill_color=colormap(row['AvgDelay']),
            fill_opacity=0.8,
            popup=folium.Popup(
                f"<b>{row['Org_Airport']} ({row['Origin']})</b><br>"
                f"Flights: {row['Count']}<br>"
                f"Avg Delay: {row['AvgDelay']:.2f} mins"
            ),
        ).add_to(m_na)

colormap.add_to(m_na)
m_na.save("busiest_airports_NorthAmerica_delay.html")
m_na
